In [30]:
# imports
import os
import requests
from bs4 import BeautifulSoup
import time
import undetected_chromedriver as uc
from dotenv import load_dotenv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import Markdown, display
import openai

In [31]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [32]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        
        # Set up undetected Chrome options
        options = uc.ChromeOptions()
        options.add_argument("--disable-gpu")
        options.add_argument("window-size=1920,1080")
        # For debugging, you can run in normal mode (i.e., not headless)
        # options.add_argument("--headless")
        
        # Initialize undetected_chromedriver
        driver = uc.Chrome(options=options,  version_main=132)
        driver.get(url)
        
        try:
            # Use an explicit wait for an element that confirms the page has fully loaded.
            # Adjust the selector to one that appears after the challenge is complete.
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
        except Exception as e:
            print("Timeout waiting for page to load:", e)
        
        # Alternatively, you can add a longer sleep if necessary
        time.sleep(10)
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Extract title and clean up unwanted tags
        self.title = driver.title if driver.title else "No title found"
        if soup.body:
            for tag in soup.body.find_all(["script", "style", "img", "input"]):
                tag.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = "No body content found."
        
        # For debugging, you might want to keep the browser open to inspect:
        # input("Press Enter to close the browser...")

        driver.quit()


In [33]:
# Testing the modified Website class
#ed = Website("https://edwarddonner.com")
ed = Website("https://openai.com")
print(ed.title)
print(ed.text)

OpenAI
Skip to main content
Log in
Switch to
ChatGPT
(opens in a new window)
Sora
(opens in a new window)
API Platform
(opens in a new window)
Research
Safety
ChatGPT
Sora
API Platform
For Business
Stories
Company
News
Research
Back to main menu
Research Index
Research Overview
Research Residency
Latest Advancements
GPT-4.5
OpenAI o3-mini
OpenAI o1
OpenAI o1-mini
GPT-4o
GPT-4o mini
Sora
Safety
Back to main menu
Safety Approach
Security & Privacy
ChatGPT
Back to main menu
Explore ChatGPT
Team
Enterprise
Education
Pricing
Download
Sora
Back to main menu
Sora Overview
Features
Pricing
Help center
(opens in a new window)
Sora log in
(opens in a new window)
API Platform
Back to main menu
Platform Overview
Pricing
API Log in
(opens in a new window)
Documentation
(opens in a new window)
Developer Forum
(opens in a new window)
For Business
Stories
Company
Back to main menu
About us
Our Charter
Careers
Brand
Overview
Logos
Gallery
Partnerships
Typography
Language
Contact
News
Log in
OpenAI
What

In [34]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown. "

In [35]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [36]:
print(user_prompt_for(ed))

You are looking at a website titled OpenAI
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Skip to main content
Log in
Switch to
ChatGPT
(opens in a new window)
Sora
(opens in a new window)
API Platform
(opens in a new window)
Research
Safety
ChatGPT
Sora
API Platform
For Business
Stories
Company
News
Research
Back to main menu
Research Index
Research Overview
Research Residency
Latest Advancements
GPT-4.5
OpenAI o3-mini
OpenAI o1
OpenAI o1-mini
GPT-4o
GPT-4o mini
Sora
Safety
Back to main menu
Safety Approach
Security & Privacy
ChatGPT
Back to main menu
Explore ChatGPT
Team
Enterprise
Education
Pricing
Download
Sora
Back to main menu
Sora Overview
Features
Pricing
Help center
(opens in a new window)
Sora log in
(opens in a new window)
API Platform
Back to main menu
Platform Overview
Pricing
API Log in
(opens in a new window)
Documentation
(opens in a new window)
Deve

In [37]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [38]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown. '},
 {'role': 'user',
  'content': 'You are looking at a website titled OpenAI\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nSkip to main content\nLog in\nSwitch to\nChatGPT\n(opens in a new window)\nSora\n(opens in a new window)\nAPI Platform\n(opens in a new window)\nResearch\nSafety\nChatGPT\nSora\nAPI Platform\nFor Business\nStories\nCompany\nNews\nResearch\nBack to main menu\nResearch Index\nResearch Overview\nResearch Residency\nLatest Advancements\nGPT-4.5\nOpenAI o3-mini\nOpenAI o1\nOpenAI o1-mini\nGPT-4o\nGPT-4o mini\nSora\nSafety\nBack to main menu\nSafety Approach\nSecurity & Privacy\nChatGPT\nBack to main menu\nExplore ChatGPT\nTeam\nEnterprise\nEducation\nPri

In [39]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

    

In [40]:
#This seems to not working - Need some help

print(summarize("https://openai.com"))


# OpenAI Website Summary

OpenAI's website provides comprehensive information about its various products and services, particularly focusing on advanced AI technologies like ChatGPT and the new tools introduced for agent building, such as GPT-4.5. 

## Key Highlights

### Products & Services
- **ChatGPT**: A conversational AI tool that assists users with a wide range of queries.
- **Sora**: A platform offering AI-driven solutions, including math tutoring and more.
- **API Platform**: Tools and documentation for developers looking to integrate OpenAI's technologies into their applications.

### Latest News and Announcements
- **Introducing GPT-4.5**: Announced on February 27, 2025, this update represents the latest iteration of their language model.
- **New Tools for Building Agents**: Launched on March 11, 2025, these tools enhance the capabilities of AI agents.
- **OpenAI Collaborations**: Recent partnerships include collaborations with the Guardian Media Group (February 14, 2025) and

In [41]:
summarize("https://openai.com")

"# OpenAI Website Summary\n\nThe OpenAI website provides information about the organization and its various products, including ChatGPT, Sora, and the API platform. It highlights the latest research initiatives, advancements in AI technology, and safety measures for AI deployment.\n\n## Key Features:\n\n- **Research Section**: Showcases an index of research efforts, including latest advancements like GPT-4.5, OpenAI o3-mini and more.\n- **Products and Tools**: Offers details on AI products including ChatGPT and Sora, highlighting their features and pricing.\n- **For Business**: Information on how businesses can utilize OpenAI's products and services.\n- **Company Information**: About OpenAI, its charter, and career opportunities.\n\n## Latest News and Announcements:\n\n1. **Introducing NextGenAI** - Announced on Mar 4, 2025.\n2. **OpenAI and Guardian Media Group Content Partnership** - Launched on Feb 14, 2025.\n3. **Data Residency in Europe** - Introduced on Feb 5, 2025.\n4. **OpenAI 

In [42]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [43]:
display_summary("https://openai.com")

# OpenAI Website Summary

OpenAI's website serves as a hub for information regarding their various products and research initiatives. It offers access to several AI tools, primarily focusing on applications like ChatGPT and Sora, an educational tool. The site features information on API services, safety protocols, and business solutions.

## Notable Announcements:

- **Introducing GPT-4.5 (Feb 27, 2025)**: A new iteration of their Generative Pre-trained Transformer (GPT) model.
- **NextGenAI Launch (Mar 4, 2025)**: A notable release in their product offerings.
- **Partnership with Guardian Media Group (Feb 14, 2025)**: OpenAI has launched a content partnership, expanding its reach in media.
- **Data Residency in Europe (Feb 5, 2025)**: New measures for data compliance within European jurisdictions.
- **OpenAI with CSU System (Feb 4, 2025)**: Collaboration to bring AI access to over 500,000 students and faculty members.

The website also provides resources on their ongoing research, safety practices, and information for developers and businesses interested in utilizing OpenAI's services.